In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import logging

In [2]:
class FatigueModelTrainer:
    def __init__(self, data_dir, img_size=(224, 224), batch_size=32):
        # Initialize logging
        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger(__name__)

        # Configuration
        self.img_size = img_size
        self.batch_size = batch_size
        self.data_dir = data_dir

        # Enable Metal backend for M1 Mac
        try:
            tf.config.experimental.set_visible_devices(
                tf.config.list_physical_devices('GPU')[0], 'GPU'
            )
        except:
            self.logger.warning("No GPU found, using CPU instead")

    def create_model(self):
        """Create the CNN model architecture using functional API"""
        # Input layer
        inputs = Input(shape=(*self.img_size, 3))

        # First Convolutional Block
        x = Conv2D(32, (3, 3), activation='relu')(inputs)
        x = MaxPooling2D(2, 2)(x)

        # Second Convolutional Block
        x = Conv2D(64, (3, 3), activation='relu')(x)
        x = MaxPooling2D(2, 2)(x)

        # Third Convolutional Block
        x = Conv2D(128, (3, 3), activation='relu')(x)
        x = MaxPooling2D(2, 2)(x)

        # Fourth Convolutional Block
        x = Conv2D(128, (3, 3), activation='relu')(x)
        x = MaxPooling2D(2, 2)(x)

        # Flatten and Dense Layers
        x = Flatten()(x)
        x = Dense(512, activation='relu')(x)
        x = Dropout(0.5)(x)
        outputs = Dense(1, activation='sigmoid')(x)

        # Create model
        model = Model(inputs=inputs, outputs=outputs)

        # Compile model
        model.compile(
            optimizer='adam',
            loss='binary_crossentropy',
            metrics=['accuracy']
        )

        return model

    def setup_data_generators(self):
        """Set up data generators for training and validation"""
        # Data augmentation for training
        train_datagen = ImageDataGenerator(
            rescale=1./255,
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            validation_split=0.2  # 20% for validation
        )

        # Create generators
        train_generator = train_datagen.flow_from_directory(
            self.data_dir,
            target_size=self.img_size,
            batch_size=self.batch_size,
            class_mode='binary',
            subset='training'
        )

        validation_generator = train_datagen.flow_from_directory(
            self.data_dir,
            target_size=self.img_size,
            batch_size=self.batch_size,
            class_mode='binary',
            subset='validation'
        )

        return train_generator, validation_generator

    def train_model(self, epochs=50):
        """Train the fatigue detection model"""
        # Create model
        model = self.create_model()
        self.logger.info("Model created successfully")

        # Setup data generators
        train_generator, validation_generator = self.setup_data_generators()
        self.logger.info("Data generators created successfully")

        # Create models directory if it doesn't exist
        os.makedirs('models', exist_ok=True)

        # Callbacks
        checkpoint = ModelCheckpoint(
            'models/best_fatigue_model.keras',  # Updated extension to .keras
            monitor='val_accuracy',
            save_best_only=True,
            mode='max',
            verbose=1
        )

        early_stopping = EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True
        )

        # Train model
        history = model.fit(
            train_generator,
            epochs=epochs,
            validation_data=validation_generator,
            callbacks=[checkpoint, early_stopping]
        )

        # Save final model
        model.save('models/final_fatigue_model.keras')  # Updated extension to .keras
        self.logger.info("Model training completed and saved")

        return history, model

    def plot_training_history(self, history):
        """Plot training history"""
        plt.figure(figsize=(12, 4))

        # Plot accuracy
        plt.subplot(1, 2, 1)
        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        plt.title('Model Accuracy')
        plt.ylabel('Accuracy')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'])

        # Plot loss
        plt.subplot(1, 2, 2)
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title('Model Loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'])

        # Create plots directory if it doesn't exist
        os.makedirs('plots', exist_ok=True)

        plt.tight_layout()
        plt.savefig('plots/training_history.png')
        plt.close()

In [3]:
if __name__ == "__main__":

    # Initialize trainer
    trainer = FatigueModelTrainer(
        data_dir='dataset',  # Make sure the 'dataset' folder is in the current directory
        img_size=(224, 224),
        batch_size=32
    )

    # Train model
    history, model = trainer.train_model(epochs=50)

    # Plot training history
    trainer.plot_training_history(history)

INFO:__main__:Model created successfully


Found 7296 images belonging to 2 classes.
Found 1824 images belonging to 2 classes.


INFO:__main__:Data generators created successfully
/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
228/228 ━━━━━━━━━━━━━━━━━━━━ 0s 866ms/step - accuracy: 0.6347 - loss: 0.6556
Epoch 1: val_accuracy improved from -inf to 0.70450, saving model to models/best_fatigue_model.keras
228/228 ━━━━━━━━━━━━━━━━━━━━ 227s 994ms/step - accuracy: 0.6349 - loss: 0.6554 - val_accuracy: 0.7045 - val_loss: 0.5611
Epoch 2/50
228/228 ━━━━━━━━━━━━━━━━━━━━ 0s 844ms/step - accuracy: 0.7376 - loss: 0.5057
Epoch 2: val_accuracy improved from 0.70450 to 0.70614, saving model to models/best_fatigue_model.keras
228/228 ━━━━━━━━━━━━━━━━━━━━ 221s 969ms/step - accuracy: 0.7376 - loss: 0.5057 - val_accuracy: 0.7061 - val_loss: 0.5627
Epoch 3/50
228/228 ━━━━━━━━━━━━━━━━━━━━ 0s 882ms/step - accuracy: 0.7578 - loss: 0.4565
Epoch 3: val_accuracy did not improve from 0.70614
228/228 ━━━━━━━━━━━━━━━━━━━━ 230s 1s/step - accuracy: 0.7578 - loss: 0.4565 - val_accuracy: 0.7045 - val_loss: 0.5835
Epoch 4/50
228/228 ━━━━━━━━━━━━━━━━━━━━ 0s 888ms/step - accuracy: 0.7713 - loss: 0.4360
Epoch 4: val_accuracy improved f

INFO:__main__:Model training completed and saved
